In [1]:
import os

In [2]:
%pwd

'/Users/more/Documents/projects/ml-ai/nlp/text-summarizer/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/more/Documents/projects/ml-ai/nlp/text-summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from text_summarizer.logging import logger
from text_summarizer.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(
            self,
            config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            file_name, headers = request.urlretrieve(
                url = self.config.source_URL, 
                filename = self.config.local_data_file
                )
            logger.info(f"Downloaded {file_name} with following headers: \n{headers}")
        else:
            logger.info(f"File {self.config.local_data_file} already exists of size: {get_size(Path(self.config.local_data_file))}")

        
    def extract_zip_file(self):
        """
        zip_file_path: Path
        Extracts the zip file into the data directory
        Returns: None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-21 13:54:57,900: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-21 13:54:57,902: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-21 13:54:57,903: INFO: common: Directory already exists: artifacts]
[2024-02-21 13:54:57,903: INFO: common: Directory already exists: artifacts/data_ingestion]
[2024-02-21 13:55:01,672: INFO: 1592671510: Downloaded artifacts/data_ingestion/data.zip with following headers: 
Connection: close
Content-Length: 4132218
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "a7d02621d22ea3b497cd45d3d37408c64466d239b1bcd91346de3a9945b8a0ea"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2FD8:396AC:417E6D:4B2AE8:65D6631D
Accept-Ranges: bytes
Date: Wed, 21 Feb 2024 20:54:59 GMT
Via: 1.1 varnish
X-Served-By: cache-phx1710073-P